In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update


https://www.geeksforgeeks.org/how-to-run-cuda-c-c-on-jupyter-notebook-in-google-colaboratory/

TO RUN CUDA C ON COLAB


In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
!nvidia-smi

Wed Jan 12 10:03:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%cu
#include <iostream>
__global__ void func()
{
		uint block_id = blockIdx.x;
		uint thread_id = threadIdx.x;
		int warp_size = warpSize;
		dim3 grids = gridDim; //number of blocks in each dimension
		uint block_dim = blockDim.y; //dimensions of block
	 
		printf("Hola dev%u %u %d %d %d\n", block_id, thread_id, warp_size, grids.y, block_dim);
}

int	main()
{
	func<<<10,5>>>();
	cudaDeviceSynchronize();
	std::cout << "Welcome To GeeksforGeeks\n";
	return 0;
}


Hola dev9 0 32 1 1
Hola dev9 1 32 1 1
Hola dev9 2 32 1 1
Hola dev9 3 32 1 1
Hola dev9 4 32 1 1
Hola dev8 0 32 1 1
Hola dev8 1 32 1 1
Hola dev8 2 32 1 1
Hola dev8 3 32 1 1
Hola dev8 4 32 1 1
Hola dev6 0 32 1 1
Hola dev6 1 32 1 1
Hola dev6 2 32 1 1
Hola dev6 3 32 1 1
Hola dev6 4 32 1 1
Hola dev4 0 32 1 1
Hola dev4 1 32 1 1
Hola dev4 2 32 1 1
Hola dev4 3 32 1 1
Hola dev4 4 32 1 1
Hola dev3 0 32 1 1
Hola dev3 1 32 1 1
Hola dev3 2 32 1 1
Hola dev3 3 32 1 1
Hola dev3 4 32 1 1
Hola dev1 0 32 1 1
Hola dev1 1 32 1 1
Hola dev1 2 32 1 1
Hola dev1 3 32 1 1
Hola dev1 4 32 1 1
Hola dev7 0 32 1 1
Hola dev7 1 32 1 1
Hola dev7 2 32 1 1
Hola dev7 3 32 1 1
Hola dev7 4 32 1 1
Hola dev0 0 32 1 1
Hola dev0 1 32 1 1
Hola dev0 2 32 1 1
Hola dev0 3 32 1 1
Hola dev0 4 32 1 1
Hola dev5 0 32 1 1
Hola dev5 1 32 1 1
Hola dev5 2 32 1 1
Hola dev5 3 32 1 1
Hola dev5 4 32 1 1
Hola dev2 0 32 1 1
Hola dev2 1 32 1 1
Hola dev2 2 32 1 1
Hola dev2 3 32 1 1
Hola dev2 4 32 1 1
Welcome To GeeksforGeeks



In [ ]:
%%cu 
#include <stdio.h>

__global__ void compute(float *x, float *y, float *z, float a) {
  int tid = blockIdx.x*blockDim.x + threadIdx.x;
  z[tid] = a*x[tid]+y[tid];
}

int main(void) {
  int N = 1024*1024;
  float *d_x, *d_y, *d_z;
  float a;
  size_t size = N * sizeof(float);
  float *x = (float*)malloc(size);
  float *y = (float*)malloc(size);
  float *z = (float*)malloc(size);
  
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);
 
  cudaMalloc((void**)&d_x, size);
  cudaMalloc((void**)&d_y, size);
  cudaMalloc((void**)&d_z, size);

  for (int k=0; k<N; k++) {
    x[k] = 0.5;
    y[k] = 1.0;
  }
  
  cudaEventRecord(start); //timer started before copying into memory
  
  cudaMemcpy(d_x, x, size, cudaMemcpyHostToDevice);
  cudaMemcpy(d_y, y, size, cudaMemcpyHostToDevice);

  a = 10.1;

  int block_size = (int) 1024;
  int thread_size = (int) 1024;
  
  //Timer can be added here too to see time taken to compute
  compute<<<block_size, thread_size>>>(x, y, z, a);

  cudaMemcpy(z, d_z, size, cudaMemcpyDeviceToHost);

  cudaEventRecord(stop); //timer ended after copying into memory

  float milliseconds = 0;
  cudaEventElapsedTime(&milliseconds, start, stop);

  for (int i = 0; i < N; i++)  {
      //printf("%d - %f\n", i, z[i]);
  }
  
  printf("Execution Time %f \n", milliseconds);  
  cudaFree(d_x);
  cudaFree(d_y);
  cudaFree(d_z);
  free(x);
  free(y);
  free(z);
}